# "Human or Robot" 
## Predicting Auction Fraud

On an auction website, human bidders are becoming increasingly frustrated with their inability to win auctions vs. their software-controlled counterparts. As a result, usage from the site's core customer base is plummeting. In order to rebuild customer happiness, the site owners need to eliminate computer generated bidding from their auctions.

The goal is to identify online auction bids that are placed by "robots", helping the site owners easily flag these users for removal from their site to prevent unfair auction activity.


<img src="../images/robots-greeting.png" width="500">

## EXPLORATORY DATA ANALYSIS

## Initial hypothesis:
1. Total number of bids: Robots might have significantly higher number of bids than humans.
2. Number of bids per auction: Robots might have higher number od bids each auction.
3. Response time: Robots might bid more often, so time between each bid will be shorter than for humans' bid.
4. Distinct IPs: Robots might bid from more diverse IP addresses.
5. Merchandise: Robots might bid more often on certain merchandises. 